In [234]:
import pandas as pd
import numpy as np
import random

def generate_item_df(num_items = 20, weight_range = (0, 10), value_range = (0, 1)):
    weights = np.random.uniform(*weight_range, num_items)
    values = np.random.uniform(*value_range, num_items)
    return pd.DataFrame({
        'Name': [str(i) for i in range(num_items)],
        'Weight': weights,
        'Value': values
    })

df = generate_item_df()
print(df.head(4))

  Name    Weight     Value
0    0  3.331509  0.150684
1    1  4.279698  0.071599
2    2  7.019322  0.955734
3    3  7.887609  0.511730


In [235]:
def genetic_algorithm(df, max_weight=30, population_size=100, num_generations=50, mutation_rate=0.2):
    weights = df["Weight"].tolist()
    values = df["Value"].tolist()
    n = len(values)
    
    # Normalise mutations, so that longer lists dont become more mutated.
    mutation_rate /= n

    average_weight = sum(weights) / n
    expected_items = max_weight / average_weight
    bit_flip_probability = expected_items / n

    # Initialise the population so that each individual has roughly the correct weight
    population = [[1 if random.random() < bit_flip_probability else 0 for _ in range(n)]
                  for _ in range(population_size)]
    
    def get_total_value(individual):
        return sum(value if individual[i] else 0 for i, value in enumerate(values))
    
    def get_total_weight(individual):
        return sum(weight if individual[i] else 0 for i, weight in enumerate(weights))

    def fitness(individual):
        total_weight = get_total_weight(individual)
        total_value = get_total_value(individual)
        if total_weight > max_weight:
            return 0
        return total_value

    def tournament_selection(population, k=3):
        return max(random.sample(population, k), key=fitness)

    def crossover(parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        return parent1[:crossover_point] + parent2[crossover_point:], parent2[:crossover_point] + parent1[crossover_point:]

    def mutate(individual):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i] # Flip bit
        return individual

    for _ in range(num_generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = new_population

    best = max(population, key=fitness)
    return df[[bool(bit) for bit in best]].copy(), get_total_value(best), get_total_weight(best)


In [236]:
def show_genetic_algorithm(df, ga_func=genetic_algorithm, n_runs=4,
                           population_size=100, num_generations=50, mutation_rate=0.2):
    for _ in range(n_runs):
        items, value, weight = ga_func(df, population_size=population_size, num_generations=num_generations, mutation_rate=mutation_rate)
        print(f"Items: {', '.join(items['Name'].tolist())}")
        print(f"Total weight: {weight:.2f}")
        print(f"Total value: {value:.2f}")
        print()


In [237]:
# Test with the original 20 items
show_genetic_algorithm(df)
%timeit genetic_algorithm(df)

Items: 2, 4, 6, 10, 13, 14, 18
Total weight: 29.87
Total value: 4.90

Items: 2, 6, 7, 10, 13, 14, 15, 18
Total weight: 28.67
Total value: 4.91

Items: 2, 8, 10, 13, 14, 15, 18
Total weight: 29.18
Total value: 4.69

Items: 2, 4, 6, 10, 13, 14, 18
Total weight: 29.87
Total value: 4.90

65.8 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [238]:
# Test with 1000 items - slow
df_1000 = generate_item_df(1000)
show_genetic_algorithm(df_1000)
timeit_1000 = %timeit -o genetic_algorithm(df_1000)

Items: 50, 79, 88, 107, 143, 184, 257, 259, 329, 453, 473, 476, 530, 549, 621, 678, 684, 739, 766, 847, 889, 902, 998
Total weight: 29.87
Total value: 16.74

Items: 38, 47, 110, 145, 178, 179, 192, 204, 259, 282, 358, 359, 395, 422, 476, 496, 506, 627, 645, 652, 684, 686, 740, 765, 782, 895, 910, 915, 947, 949, 950, 967
Total weight: 30.00
Total value: 23.62

Items: 5, 79, 80, 155, 181, 190, 204, 259, 285, 328, 358, 402, 473, 476, 524, 530, 603, 638, 645, 671, 740, 794, 864, 902, 932, 977
Total weight: 30.00
Total value: 18.65

Items: 143, 151, 220, 222, 225, 296, 311, 394, 447, 473, 476, 556, 634, 652, 657, 664, 686, 740, 755, 902, 932, 961, 979
Total weight: 29.73
Total value: 16.89

2.28 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [241]:
# Rewrite using numpy
def genetic_algorithm_np(df, max_weight=30, population_size=100, num_generations=50, mutation_rate=0.2):
    weights = df["Weight"].to_numpy()
    values = df["Value"].to_numpy()
    n = len(values)
    
    # Normalise mutations, so that longer lists dont become more mutated.
    mutation_rate /= n

    average_weight = np.mean(weights)
    expected_items = max_weight / average_weight
    bit_flip_probability = expected_items / n

    # Initialise the population so that each individual has roughly the correct weight
    population = np.random.rand(population_size, n) < bit_flip_probability
    
    def fitness(individual):
        total_weight = np.dot(weights, individual)
        total_value = np.dot(values, individual)
        return total_value if total_weight <= max_weight else 0

    def tournament_selection(population, k=3):
        return max(random.sample(list(population), k), key=fitness)

    def crossover(parent1, parent2):
        crossover_point = random.randint(1, n - 1)
        offspring1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        offspring2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        return offspring1, offspring2

    def mutate(individual):
        mutation_mask = np.random.rand(n) < mutation_rate
        individual[mutation_mask] = 1 - individual[mutation_mask] # Flip bit
        return individual

    for _ in range(num_generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = np.array(new_population)

    best = max(population, key=fitness)
    best_items = df.iloc[best.astype(bool)].copy()
    return best_items, np.dot(values, best), np.dot(weights, best)

In [242]:
# Test numpy version - 10x faster
show_genetic_algorithm(df_1000, ga_func=genetic_algorithm_np)
timeit_1000_fast = %timeit -o genetic_algorithm_np(df_1000)
print(f"{timeit_1000.best/timeit_1000_fast.best:.2f}x faster")

Items: 30, 44, 48, 179, 222, 296, 329, 345, 473, 546, 549, 614, 634, 638, 645, 766, 770, 782, 877, 924, 950
Total weight: 29.90
Total value: 16.23

Items: 30, 80, 107, 251, 259, 295, 329, 343, 359, 383, 513, 652, 686, 748, 765, 782, 789, 819, 850, 932, 977
Total weight: 29.94
Total value: 17.05

Items: 46, 47, 80, 109, 178, 235, 256, 257, 310, 359, 422, 443, 513, 519, 538, 619, 634, 674, 741, 748, 915, 920, 979
Total weight: 29.98
Total value: 16.63

Items: 143, 145, 157, 178, 269, 286, 318, 473, 476, 504, 551, 603, 638, 686, 707, 752, 768, 813, 865, 905, 930, 948, 972
Total weight: 29.88
Total value: 15.71

232 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.76x faster


In [260]:
# Try different mutation rates
n_runs = 10
for mutation_rate in (0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100):
    _, values, weights = list(zip(*[genetic_algorithm_np(df_1000, mutation_rate=mutation_rate) for _ in range(n_runs)]))
    print(f"Mutation rate: {mutation_rate}".ljust(22) + 
        f"Max weight: {max(weights):.3f}  "
        f"Average value: {sum(values)/n_runs:.3f}")

Mutation rate: 0      Max weight: 29.995  Average value: 10.640
Mutation rate: 0.001  Max weight: 29.950  Average value: 10.663
Mutation rate: 0.005  Max weight: 29.984  Average value: 11.777
Mutation rate: 0.01   Max weight: 29.964  Average value: 12.320
Mutation rate: 0.05   Max weight: 29.972  Average value: 12.791
Mutation rate: 0.1    Max weight: 29.984  Average value: 16.518
Mutation rate: 0.5    Max weight: 29.940  Average value: 20.379
Mutation rate: 1      Max weight: 29.901  Average value: 19.782
Mutation rate: 5      Max weight: 1048.483  Average value: 102.258
Mutation rate: 10     Max weight: 1681.427  Average value: 160.254
Mutation rate: 50     Max weight: 2513.048  Average value: 251.938
Mutation rate: 100    Max weight: 2550.751  Average value: 248.587


- It looks like somewhere between the mutation rates of 1 and 5 (expected flips per individual), every member of the population becomes overweight.
- 0.5 - 1 looks optimal (for n = 1000 at least).